<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_5SSC_Experiment/blob/main/SleepStage_EEG_EXperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Function

In [3]:
!pip install imbalanced-learn
!pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00


In [4]:
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [5]:
def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [6]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
"""from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])"""

(108920, 52)
(108450, 52)


'from sklearn.preprocessing import LabelEncoder\nencoder=LabelEncoder()\ndataset[target]=encoder.fit_transform(dataset[target])'

###Spliting into X and y

In [11]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

#Feature Selection

In [ ]:
number_of_feat = 30

###ANOVA with f classifciation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])

###Embedded Method

In [ ]:
''''
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
'''

###Pearson's with f regression

In [ ]:
'''
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
'''

###Sequential Feature Selection

In [ ]:
''''
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
  '''

###Feature list

In [ ]:
dfcolumns = pd.DataFrame(list_of_feat)
print(dfcolumns)

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [45]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [46]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [47]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [48]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [49]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [50]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [52]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [54]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [55]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [56]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [57]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [58]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [59]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [60]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  "Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  "Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  "Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  "Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  "Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  "Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_13()
}

In [63]:
list(tune_dic.keys())

['Smote : NO    ; Scaling : NO',
 'Smote : NO    ; Scaling : All Data(Standard)',
 'Smote : NO    ; Scaling : All Data(MinMax)',
 'Smote : NO    ; Scaling : Train(Standard)',
 'Smote : NO    ; Scaling : Train(MinMax)',
 'Smote : ALL   ; Scaling : NO',
 'Smote : Train ; Scaling : NO',
 'Smote : Train ; Scaling : Train(Standard)',
 'Smote : Train ; Scaling : Train(MinMax)',
 'Smote : All   ; Scaling : Train(Standard)',
 'Smote : All   ; Scaling : Train(MinMax)',
 'Smote : All   ; Scaling : All(Standard)']



---



# **Training**

##ADABOOST

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier

ada_defult = AdaBoostClassifier(random_state=0)
ada_defult.fit(X_train, y_train)
y_pred = ada_defult.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(ada_defult,1,'AdaBoostClassifier')]=accuracy_score(y_test, y_pred)


[[ 127  829  102  544  783]
 [  72 4755 1668  772  603]
 [  11 1021 3119   16   52]
 [  86 1412  161 1435  306]
 [  76  527   74  214 2925]]
              precision    recall  f1-score   support

           0       0.34      0.05      0.09      2385
           1       0.56      0.60      0.58      7870
           2       0.61      0.74      0.67      4219
           3       0.48      0.42      0.45      3400
           4       0.63      0.77      0.69      3816

    accuracy                           0.57     21690
   macro avg       0.52      0.52      0.50     21690
weighted avg       0.54      0.57      0.54     21690

Accurecy:  0.5698939603503919
CPU times: user 18.9 s, sys: 36 ms, total: 19 s
Wall time: 18.9 s




---



---



##Graddient Boosting

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier

gradBoost_default = GradientBoostingClassifier(random_state=0)
gradBoost_default.fit(X_train, y_train)
y_pred = gradBoost_default.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(gradBoost_default,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[ 369  883   54  415  664]
 [ 166 5955  852  519  378]
 [   1 1123 3044   14   37]
 [ 187 1026   99 1930  158]
 [ 205  452   33   78 3048]]
              precision    recall  f1-score   support

           0       0.40      0.15      0.22      2385
           1       0.63      0.76      0.69      7870
           2       0.75      0.72      0.73      4219
           3       0.65      0.57      0.61      3400
           4       0.71      0.80      0.75      3816

    accuracy                           0.66     21690
   macro avg       0.63      0.60      0.60     21690
weighted avg       0.65      0.66      0.64     21690

Accurecy:  0.6614107883817427
CPU times: user 8min 12s, sys: 529 ms, total: 8min 13s
Wall time: 8min 11s


###Histogram-Based Gradient Boosting

In [ ]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)
hisgradBoost_default.fit(X_train, y_train)
y_pred = hisgradBoost_default.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(hisgradBoost_default,3,'HistGradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[ 497  824   38  380  646]
 [ 200 6229  736  402  303]
 [   9  940 3241    7   22]
 [ 165  744   52 2335  104]
 [ 226  293   19   56 3222]]
              precision    recall  f1-score   support

           0       0.45      0.21      0.29      2385
           1       0.69      0.79      0.74      7870
           2       0.79      0.77      0.78      4219
           3       0.73      0.69      0.71      3400
           4       0.75      0.84      0.79      3816

    accuracy                           0.72     21690
   macro avg       0.68      0.66      0.66     21690
weighted avg       0.70      0.72      0.70     21690

Accurecy:  0.71572153065929
CPU times: user 33.5 s, sys: 265 ms, total: 33.7 s
Wall time: 17.6 s




---



---



##Random Forest

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)
rf_default.fit(X_train, y_train)
y_pred=rf_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_default,4,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

[[ 550  817   33  347  638]
 [ 225 6383  607  378  277]
 [   5  875 3307    6   26]
 [ 159  763   42 2316  120]
 [ 247  320   19   46 3184]]
              precision    recall  f1-score   support

           0       0.46      0.23      0.31      2385
           1       0.70      0.81      0.75      7870
           2       0.83      0.78      0.80      4219
           3       0.75      0.68      0.71      3400
           4       0.75      0.83      0.79      3816

    accuracy                           0.73     21690
   macro avg       0.70      0.67      0.67     21690
weighted avg       0.71      0.73      0.71     21690

Accurecy:  0.7256800368833564
CPU times: user 1min 31s, sys: 83.9 ms, total: 1min 31s
Wall time: 47 s




---



---



##XGB

In [ ]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)
xgb_deafult.fit(X_train.values,y_train.values)
y_pred = xgb_deafult.predict(X_test.values)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_deafult,5,'xgboost')]=accuracy_score(y_test, y_pred)

[[ 263  879   64  453  726]
 [ 130 5833  934  568  405]
 [   2 1214 2949   11   43]
 [ 151 1113  116 1835  185]
 [ 155  464   41   83 3073]]
              precision    recall  f1-score   support

           0       0.38      0.11      0.17      2385
           1       0.61      0.74      0.67      7870
           2       0.72      0.70      0.71      4219
           3       0.62      0.54      0.58      3400
           4       0.69      0.81      0.75      3816

    accuracy                           0.64     21690
   macro avg       0.60      0.58      0.57     21690
weighted avg       0.62      0.64      0.62     21690

Accurecy:  0.6432918395573998
CPU times: user 1min 53s, sys: 108 ms, total: 1min 53s
Wall time: 59.2 s


##KNN

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)
knn_default.fit(X_train, y_train)
y_pred=knn_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(knn_default,6,'KNeighborsClassifier')]=accuracy_score(y_test, y_pred)

[[ 567  895  121  346  456]
 [ 582 5348 1113  621  206]
 [  41 1281 2820   54   23]
 [ 412 1548  173 1127  140]
 [ 529  485  119  190 2493]]
              precision    recall  f1-score   support

           0       0.27      0.24      0.25      2385
           1       0.56      0.68      0.61      7870
           2       0.65      0.67      0.66      4219
           3       0.48      0.33      0.39      3400
           4       0.75      0.65      0.70      3816

    accuracy                           0.57     21690
   macro avg       0.54      0.51      0.52     21690
weighted avg       0.57      0.57      0.56     21690

Accurecy:  0.5696173351775011
CPU times: user 1min 9s, sys: 3.66 s, total: 1min 13s
Wall time: 55.9 s


##NB

In [ ]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()
nb_deafult.fit(X_train, y_train)
y_pred = nb_deafult.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(nb_deafult,7,'GaussianNB')]=accuracy_score(y_test, y_pred)

[[  50 2335    0    0    0]
 [ 121 7749    0    0    0]
 [  47 4172    0    0    0]
 [  28 3372    0    0    0]
 [  46 3770    0    0    0]]
              precision    recall  f1-score   support

           0       0.17      0.02      0.04      2385
           1       0.36      0.98      0.53      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.00      0.00      0.00      3816

    accuracy                           0.36     21690
   macro avg       0.11      0.20      0.11     21690
weighted avg       0.15      0.36      0.20     21690

Accurecy:  0.3595666205624712
CPU times: user 161 ms, sys: 20 ms, total: 181 ms
Wall time: 178 ms


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Support Vector Machines

####SVM

In [ ]:
"""%%time
from sklearn import svm

svm = svm.SVC(decision_function_shape='ovo')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(svm,8,'svm.SVC')]=accuracy_score(y_test, y_pred)"""

[[   0 2385    0    0    0]
 [   0 7870    0    0    0]
 [   0 4219    0    0    0]
 [   0 3400    0    0    0]
 [   0 3816    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2385
           1       0.36      1.00      0.53      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.00      0.00      0.00      3816

    accuracy                           0.36     21690
   macro avg       0.07      0.20      0.11     21690
weighted avg       0.13      0.36      0.19     21690

Accurecy:  0.3628400184416782
CPU times: user 19min 55s, sys: 1.23 s, total: 19min 57s
Wall time: 19min 51s


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####Linear

In [ ]:
"""%%time
from sklearn import svm

LinearSVC = svm.LinearSVC()
LinearSVC.fit(X_train, y_train)
y_pred = LinearSVC.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(LinearSVC,9,'svm.LinearSVC')]=accuracy_score(y_test, y_pred)"""

##Stochastic Gradient Descent

In [ ]:
"""%%time
from sklearn.linear_model import SGDClassifier

SGDClassifier = SGDClassifier()
SGDClassifier.fit(X_train, y_train)
y_pred = SGDClassifier.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(SGDClassifier,10,'SGDClassifier')]=accuracy_score(y_test, y_pred)"""

[[   0    0    0    0 2385]
 [   0    0    0    0 7870]
 [   0    0    0    0 4219]
 [   0    0    0    0 3400]
 [   0    0    0    0 3816]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2385
           1       0.00      0.00      0.00      7870
           2       0.00      0.00      0.00      4219
           3       0.00      0.00      0.00      3400
           4       0.18      1.00      0.30      3816

    accuracy                           0.18     21690
   macro avg       0.04      0.20      0.06     21690
weighted avg       0.03      0.18      0.05     21690

Accurecy:  0.17593360995850624
CPU times: user 13.6 s, sys: 78 ms, total: 13.7 s
Wall time: 13.6 s


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Tree algorithms

###DecisionTreeRegressor

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(dtc,11,'DecisionTreeClassifier')]=accuracy_score(y_test, y_pred)

[[ 601  741   77  406  560]
 [ 721 4960 1016  787  386]
 [  82 1057 2943   77   60]
 [ 431  830   82 1895  162]
 [ 597  440   60  175 2544]]
              precision    recall  f1-score   support

           0       0.25      0.25      0.25      2385
           1       0.62      0.63      0.62      7870
           2       0.70      0.70      0.70      4219
           3       0.57      0.56      0.56      3400
           4       0.69      0.67      0.68      3816

    accuracy                           0.60     21690
   macro avg       0.56      0.56      0.56     21690
weighted avg       0.60      0.60      0.60     21690

Accurecy:  0.596726602120793
CPU times: user 7.24 s, sys: 8.99 ms, total: 7.25 s
Wall time: 7.25 s




---



---



#Result

In [ ]:

models=[]

for i in result:
  models.append(i[0])
  print(i[0],i[1]," : ",result[i])
  print("---------------------------------------------------------------")
  print()


AdaBoostClassifier(random_state=0) 1  :  0.5698939603503919
---------------------------------------------------------------

GradientBoostingClassifier(random_state=0) 2  :  0.6614107883817427
---------------------------------------------------------------

HistGradientBoostingClassifier(random_state=0) 2  :  0.71572153065929
---------------------------------------------------------------

RandomForestClassifier(n_jobs=-1, random_state=0) 3  :  0.7256800368833564
---------------------------------------------------------------

XGBClassifier(n_jobs=-1, objective='multi:softprob') 4  :  0.6432918395573998
---------------------------------------------------------------

KNeighborsClassifier(n_jobs=-1) 5  :  0.5696173351775011
---------------------------------------------------------------

GaussianNB() 6  :  0.3595666205624712
---------------------------------------------------------------

